<center> <h1>文本数据分析</h1> </center>

#### 学习目标
- 了解文本数据分析的作用
- 掌握常用的几种文本数据分析方法

#### 文本数据分析的作用
- 文本数据分析能够有效帮助我们理解数据语料，快速检查出语料可能存在的问题，并指导之后模型训练过程中一些超参数的选择

#### 常用的几种文本数据分析方法
- 标签数量分布
- 句子长度分布
- 词频统计与关键词词云

In [ ]:
# 导入必备的工具包
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import jieba

# 设置显示风格
plt.style.use('fivethirtyeight')

# 利用pandas读取训练数据和验证数据
train_data = pd.read_csv("./cn_data/train.tsv", sep="\t")
valid_data = pd.read_csv("./cn_data/dev.tsv", sep="\t")

# 获得训练数据标签的数量分布
sns.countplot("label", data=train_data)
plt.title("train_data")
plt.show()

# 获得验证数据标签的数量分布
sns.countplot("label", data=valid_data)
plt.title("valid_data")
plt.show()

#### 分析

- 在深度学习模型评估中，我们一般使用ACC作为评估指标，若想将ACC的基线定义在50%左右，则需要我们的正负样本比例维持在1:1左右，否则就要进行必要的数据增加或数据删减，上面图中训练和验证集正负样本都稍有不均衡，可以进行一些数据增强

#### 获取训练集和验证集的句子长度分布

In [ ]:
# 在训练数据中添加新的句子长度列，每个元素的值都是对应的句子列的长度
train_data["sentence_length"] = list(map(lambda x:len(x), train_data["sentence"]))

# 绘制句子长度列的数量分布图
sns.countplot("sentence_length", data=train_data)
# 主要关注count长度分布的纵坐标，不需要绘制横坐标，横坐标范围通过dist图进行查看
plt.xticks([])
plt.show()

# 绘制dist长度分布图
sns.displot(train_data["sentence_length"])

# 主要关注dist长度分布横坐标，不需要绘制纵坐标
plt.yticks([])
plt.show()

# 在验证数据中添加新的句子长度列，每个元素的值都是对应的句子列的长度
valid_data["sentence_length"] = list(map(lambda x :len(x), valid_data["sentence"]))

# 绘制句子长度列的数量分布图
sns.countplot("sentence_length", data=valid_data)

# 主要关注count长度分布的纵坐标，不需要绘制横坐标，横坐标范围通过dist图进行查看
plt.xticks([])
plt.show()

# 绘制dist长度分布图
sns.displot(valid_data["sentence_length"])

# 主要关注count长度分布的横坐标，不需要绘制纵坐标
plt.xticks([])
plt.show()

#### 分析
- 通过绘制句子长度分布图，可以得知我们的语料中大部分句子长度的分布范围，因为模型的输入要求为固定尺寸的张量，合理的长度范围对之后进行句子的截断补齐(规范长度)起到关键的指导作用，上图中大部分句子长度的范围大致在20-250之间

#### 获取训练集和验证集的正负样本长度散点分布

In [ ]:
# 绘制训练集长度分布的散点图
sns.stripplot(y='sentence_length', x='label', data=train_data)
plt.show()

# 绘制验证集长度分布的散点图
sns.stripplot(y='sentence_length', x='label', data=valid_data)
plt.show()

#### 分析
- 通过查看正负样本长度散点图，可以有效定位异常点的出现位置，帮助我们更准确进行人式语料审查，上图中在训练集正样本中出现了异常点，它的句子长度近3500左右，需要我们人工审查

#### 获得训练集与验证集不同词汇总数的统计

In [ ]:
# 导入jieba用于分词
# 导入chain方法用于扁平化列表
import jieba
from  itertools import chain

# 进行训练集的句子进行分词，并统计出不同词汇的总数
train_vocab = set(chain(*map(lambda x: jieba.lcut(x), train_data['sentence'])))
print("训练集共包含不同词汇总数为:", len(train_vocab))

# 进行验证集的句子进行分词，并统计出不同词汇的总数
valid_vocab = set(chain(*map(lambda x: jieba.lcut(x), valid_data['sentence'])))
print("训练集共包含不同词汇总数为:", len(valid_vocab))

#### 获得训练集上正负样本的高频形容词词云

In [ ]:
# 使用jieba中的词性标注功能
import jieba.posseg as pseg

def get_a_list(text):
    """用于获取形容词列表"""
    # 使用jieba的词性标注方法切分文本，获得具有词性属性flag和词汇属性word的对象
    # 从而判断flag是否为形容词，来返回对应的词汇
    r = []
    for g in pseg.lcut(text):
        if g.flag == "a":
            r.append(g.word)
    return r

# 导入绘制词云的工具包
from wordcloud import WordCloud

def get_word_cloud(keywords_list):
    # 实例化绘制词云的类，其中参数font_path是字体路径，为了能够显示中文
    # max_words指词云图像最多显示多少个词，background_color为背影颜色
    wordcloud = WordCloud(font_path="./SimHei.ttf", max_words=100, background_color="white")
    # 将传入的列表转化成词云生成器需要的字符串形式
    keywords_string = " ".join(keywords_list)
    # 生成词云
    wordcloud.generate(keywords_string)

    # 绘制图像并显示
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


# 获得训练集上正样本
p_train_data = train_data[train_data["label"] == 1]["sentence"]

# 获得正样本的每个句子的形容词
train_p_a_vocab = chain(*map(lambda x :get_a_list(x), p_train_data))
# print(train_p_a_vocab)

# 获得训练集上负样本
n_train_data = train_data[train_data["label"] == 0]["sentence"]

# 获得负样本的每个句子的形容词
train_n_a_vocab = chain(*map(lambda x :get_a_list(x), n_train_data))

# 调用绘制词云函数
get_word_cloud(train_p_a_vocab)
get_word_cloud(train_n_a_vocab)

In [ ]:
# 获得验证集上正样本
p_valid_data = valid_data[valid_data["label"] == 1]["sentence"]

# 获得正样本的每个句子的形容词
valid_p_a_vocab = chain(*map(lambda x :get_a_list(x), p_valid_data))
# print(valid_p_a_vocab)

# 获得验证集上负样本
n_valid_data = valid_data[valid_data["label"] == 0]["sentence"]

# 获得负样本的每个句子的形容词
valid_n_a_vocab = chain(*map(lambda x :get_a_list(x), n_valid_data))

# 调用绘制词云函数
get_word_cloud(valid_p_a_vocab)
get_word_cloud(valid_n_a_vocab)

#### 分析
- 根据高频形容词词云显示，我们可以对当前语料质量进行简单评估，同时对违反语料标签含义的词汇进行人工审查和修正，来保证绝大多数的语料符合训练标准，上图中的正样本大多数是褒义词，而负样本大多数是贬义词，基本符合要求，但是负样本词云中也存在"便利"这样的褒义词，因此可以进行人工审查

#### 小结
- 学习文本数据分析的作用
	- 文本数据分析能够有效帮助我们理解数据语料，快速检查语料可能存在的问题，并指导之后模型训练过程中一些超参数的选择

- 学习常用的几种文本数据分析方法
	- 标签数量分布
	- 句子长度分布
	- 词频统计与关键词词云
- 学习基于真实的中文酒店评论语料进行几种文本数据分析方法
	- 获得训练集和验证集标签数量分布
	- 获得训练集和验证集的句子长度分布
	- 获得训练集和验证集正负样本长度散点分布
	- 获得训练集和验证集不同词汇总数统计
	- 获得训练集和验证集正负样本的高频形容词词云

In [ ]:
# 导入必备的工具包
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import jieba

# 设置显示风格
plt.style.use('fivethirtyeight')

# 利用pandas读取训练数据和验证数据
train_data = pd.read_csv("./cn_data/train.tsv", sep="\t")
valid_data = pd.read_csv("./cn_data/dev.tsv", sep="\t")

# 获得训练数据标签的数量分布
# sns.countplot("label", data=train_data)
# plt.title("train_data")
# plt.show()

# 获得验证数据标签的数量分布
# sns.countplot("label", data=valid_data)
# plt.title("valid_data")
# plt.show()

# 在训练数据中添加新的句子长度列, 每个元素的值都是对应句子的长度
train_data["sentence_length"] = list(map(lambda x: len(x), train_data["sentence"]))

# 绘制句子长度列的数量分布
# sns.countplot("sentence_length", data=train_data)
# plt.xticks([])
# plt.show()

# 绘制dist长度分布图
# sns.distplot(train_data["sentence_length"])
# plt.yticks([])
# plt.show()

# 在验证数据中添加新的句子长度列, 每个元素的值对应句子的长度
valid_data["sentence_length"] = list(map(lambda x: len(x), valid_data["sentence"]))

# 绘制句子长度列的数量分布图
# sns.countplot("sentence_length", data=valid_data)
# plt.xticks([])
# plt.show()

# 绘制dist长度分布图
# sns.distplot(valid_data["sentence_length"])
# plt.yticks([])
# plt.show()


# 绘制训练数据语句长度的散点图
# sns.stripplot(y="sentence_length", x="label", data=train_data)
# plt.show()

# 绘制验证数据语句长度的散点图
# sns.stripplot(y="sentence_length", x="label", data=valid_data)
# plt.show()


# 导入jieba 工具包和chain工具包, 用于分词扁平化列表
from itertools import chain

# 进行训练集的句子进行分词, 并统计出不同词汇的总数
# train_vocab = set(chain(*map(lambda x: jieba.lcut(x), train_data["sentence"])))
# print("训练集共包含不同词汇总数为:", len(train_vocab))

# 进行验证集的句子进行分词, 并统计出不同词汇的总数
# valid_vocab = set(chain(*map(lambda x: jieba.lcut(x), valid_data["sentence"])))
# print("验证集共包含不同词汇总数为:", len(valid_vocab))


# 导入jieba 中的词性标注工具包
import jieba.posseg as pseg

# 定义获取形容词的列表函数
def get_a_list(text):
    # 使用jieba的词性标注方法来切分文本, 获得两个属性word,flag
    # 利用flag属性去判断一个词汇是否是形容词
    r = []
    for g in pseg.lcut(text):
        if g.flag == 'a':
            r.append(g.word)
    return r


# 导入绘制词云的工具包
from wordcloud import WordCloud

# 定义获取词云的函数并绘图
def get_word_cloud(keywords_list):
    # 首先实例化词云类对象, 里面三个参数
    # font_path: 字体路径,为了能够更好的显示中文
    # max_words: 指定词云图像最多可以显示的词汇数量
    # backgroud_color: 代表图片的北京颜色
    wordcloud = WordCloud(max_words=100, background_color='white')

    # 将传入的列表参数转化为字符串形式, 因为词云对象的参数要求是字符串类型
    keywords_string = " ".join(keywords_list)
    # 生成词云
    wordcloud.generate(keywords_string)

    # 绘图
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


# 获取训练集上的正样本
# p_train_data = train_data[train_data["label"]==1]["sentence"]

# 对正样本的每个句子提取形容词
# train_p_a_vocab = chain(*map(lambda x: get_a_list(x), p_train_data))

# 获取训练集上的负样本
# n_train_data = train_data[train_data["label"]==0]["sentence"]

# 对负样本的每个句子提取形容词
# train_n_a_vocab = chain(*map(lambda x: get_a_list(x), n_train_data))

# 调用获取词云的函数
# get_word_cloud(train_p_a_vocab)
# get_word_cloud(train_n_a_vocab)


# 获取验证集的数据正样本
p_valid_data = valid_data[valid_data["label"]==1]["sentence"]

# 获取正样本的每个句子的形容词
valid_p_a_vocab = chain(*map(lambda x: get_a_list(x), p_valid_data))

# 获取验证集的数据负样本
n_valid_data = valid_data[valid_data["label"]==0]["sentence"]

# 获取负样本的每个句子的形容词
valid_n_a_vocab = chain(*map(lambda x: get_a_list(x), n_valid_data))

# 调用获取词云的函数
get_word_cloud(valid_p_a_vocab)
get_word_cloud(valid_n_a_vocab)